In [1]:
"""
Tests of neo.rawio.spikeglxrawio
"""

import unittest

from neo.rawio.spikeglxrawio import SpikeGLXRawIO
from neo.test.rawiotest.common_rawio_test import BaseTestRawIO
import numpy as np


In [2]:


class TestSpikeGLXRawIO(BaseTestRawIO, unittest.TestCase):
    rawioclass = SpikeGLXRawIO
    entities_to_download = ["spikeglx"]
    entities_to_test = [
        "spikeglx/Noise4Sam_g0",
        "spikeglx/TEST_20210920_0_g0",
        # this is only g0 multi index
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI0/5-19-2022-CI0_g0",
        # this is only g1 multi index
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI0/5-19-2022-CI0_g1",
        # this mix both multi gate and multi trigger (and also multi probe)
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI0",
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI1",
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI2",
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI3",
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI4",
        "spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI5",
        # different sync/sybset options with commercial NP2
        "spikeglx/NP2_with_sync",
        "spikeglx/NP2_no_sync",
        "spikeglx/NP2_subset_with_sync",
        # NP-ultra
        "spikeglx/np_ultra_stub",
        # Filename changed by the user, multi-dock
        "spikeglx/multi_probe_multi_dock_multi_shank_filename_without_info",
        # CatGT
        "spikeglx/multi_trigger_multi_gate/CatGT/CatGT-A",
        "spikeglx/multi_trigger_multi_gate/CatGT/CatGT-B",
        "spikeglx/multi_trigger_multi_gate/CatGT/CatGT-C",
        "spikeglx/multi_trigger_multi_gate/CatGT/CatGT-D",
        "spikeglx/multi_trigger_multi_gate/CatGT/CatGT-E",
        "spikeglx/multi_trigger_multi_gate/CatGT/Supercat-A",
    ]

    def test_with_location(self):
        rawio = SpikeGLXRawIO(self.get_local_path("spikeglx/Noise4Sam_g0"), load_channel_location=True)
        rawio.parse_header()
        # one of the stream have channel location
        have_location = []
        for sig_anotations in rawio.raw_annotations["blocks"][0]["segments"][0]["signals"]:
            have_location.append("channel_location_0" in sig_anotations["__array_annotations__"])
        assert any(have_location)

    def test_sync(self):
        rawio_with_sync = SpikeGLXRawIO(self.get_local_path("spikeglx/NP2_with_sync"), load_sync_channel=True)
        rawio_with_sync.parse_header()
        stream_index = list(rawio_with_sync.header["signal_streams"]["name"]).index("imec0.ap")

        # AP stream has 385 channels
        chunk = rawio_with_sync.get_analogsignal_chunk(
            block_index=0, seg_index=0, i_start=0, i_stop=100, stream_index=stream_index
        )
        assert chunk.shape[1] == 385

        rawio_no_sync = SpikeGLXRawIO(self.get_local_path("spikeglx/NP2_with_sync"), load_sync_channel=False)
        rawio_no_sync.parse_header()

        # AP stream has 384 channels
        chunk = rawio_no_sync.get_analogsignal_chunk(
            block_index=0, seg_index=0, i_start=0, i_stop=100, stream_index=stream_index
        )
        assert chunk.shape[1] == 384

    def test_no_sync(self):
        # requesting sync channel when there is none raises an error
        with self.assertRaises(ValueError):
            rawio_no_sync = SpikeGLXRawIO(self.get_local_path("spikeglx/NP2_no_sync"), load_sync_channel=True)
            rawio_no_sync.parse_header()

    def test_subset_with_sync(self):
        rawio_sub = SpikeGLXRawIO(self.get_local_path("spikeglx/NP2_subset_with_sync"), load_sync_channel=True)
        rawio_sub.parse_header()
        stream_index = list(rawio_sub.header["signal_streams"]["name"]).index("imec0.ap")

        # AP stream has 121 channels
        chunk = rawio_sub.get_analogsignal_chunk(
            block_index=0, seg_index=0, i_start=0, i_stop=100, stream_index=stream_index
        )
        assert chunk.shape[1] == 121

        rawio_sub_no_sync = SpikeGLXRawIO(self.get_local_path("spikeglx/NP2_subset_with_sync"), load_sync_channel=False)
        rawio_sub_no_sync.parse_header()
        # AP stream has 120 channels
        chunk = rawio_sub_no_sync.get_analogsignal_chunk(
            block_index=0, seg_index=0, i_start=0, i_stop=100, stream_index=stream_index
        )
        assert chunk.shape[1] == 120

    def test_nidq_digital_channel(self):
        rawio_digital = SpikeGLXRawIO(self.get_local_path("spikeglx/DigitalChannelTest_g0"))
        rawio_digital.parse_header()
        # This data should have 8 event channels
        assert np.shape(rawio_digital.header["event_channels"])[0] == 8

        # Channel 0 in this data will have sync pulses at 1 Hz, let's confirm that
        all_events = rawio_digital.get_event_timestamps(0, 0, 0)
        on_events = np.where(all_events[2] == "XD0 ON")
        on_ts = all_events[0][on_events]
        on_ts_scaled = rawio_digital.rescale_event_timestamp(on_ts)
        on_diff = np.diff(on_ts_scaled)
        atol = 0.001
        assert np.allclose(on_diff, 1, atol=atol)

    def test_t_start_reading(self):
        """Test that t_start values are correctly read for all streams and segments."""

        # Expected t_start values for each stream and segment
        expected_t_starts = {
            "imec0.ap": {0: 15.319535472007237, 1: 15.339535431281986, 2: 21.284723325294053, 3: 21.3047232845688},
            "imec1.ap": {0: 15.319554693264516, 1: 15.339521518106308, 2: 21.284735282142822, 3: 21.304702106984614},
            "imec0.lf": {0: 15.3191688060872, 1: 15.339168765361949, 2: 21.284356659374016, 3: 21.304356618648765},
            "imec1.lf": {0: 15.319321358082725, 1: 15.339321516521915, 2: 21.284568614155827, 3: 21.30456877259502},
        }

        # Initialize the RawIO
        rawio = SpikeGLXRawIO(self.get_local_path("spikeglx/multi_trigger_multi_gate/SpikeGLX/5-19-2022-CI4"))
        rawio.parse_header()

        # Get list of stream names
        stream_names = rawio.header["signal_streams"]["name"]

        # Test t_start for each stream and segment
        for stream_name, expected_values in expected_t_starts.items():
            # Get stream index
            stream_index = list(stream_names).index(stream_name)

            # Check each segment
            for seg_index, expected_t_start in expected_values.items():
                actual_t_start = rawio.get_signal_t_start(block_index=0, seg_index=seg_index, stream_index=stream_index)

                # Use numpy.testing for proper float comparison
                np.testing.assert_allclose(
                    actual_t_start,
                    expected_t_start,
                    rtol=1e-9,
                    atol=1e-9,
                    err_msg=f"Mismatch in t_start for stream '{stream_name}', segment {seg_index}",
                )


In [74]:
testclass = TestSpikeGLXRawIO()

### NP2 with sync

In [90]:
# rawio = SpikeGLXRawIO(testclass.get_local_path("spikeglx/NP2_with_sync"), load_sync_channel=True)
# /home/dgupta/data/20250305-133614
rawio = SpikeGLXRawIO(testclass.get_local_path("/home/dgupta/data/20250305-133614"), load_sync_channel=True)

rawio.parse_header()
stream_index = list(rawio.header["signal_streams"]["name"]).index("imec0.ap")



In [116]:
stream_index

1

In [91]:
rawio.signals_info_dict.keys()


dict_keys([(0, 'nidq'), (0, 'imec0.ap')])

In [92]:
#finding sync channel in imec0.ap

for key in rawio.signals_info_dict[(0,'imec0.ap')]['meta']:
    if 'sync' in key:
        print(key, rawio.signals_info_dict[(0,'imec0.ap')]['meta'][key])

syncImInputSlot 2
syncSourceIdx 3
syncSourcePeriod 1


In [93]:
rawio._buffer_descriptions

{0: {0: {'nidq': {'type': 'raw',
    'file_path': '/home/dgupta/data/20250305-133614/sglx_20250305-133614.nidq.bin',
    'dtype': 'int16',
    'order': 'C',
    'file_offset': 0,
    'shape': (8022991, 2)},
   'imec0.ap': {'type': 'raw',
    'file_path': '/home/dgupta/data/20250305-133614/sglx_20250305-133614.imec0.ap.bin',
    'dtype': 'int16',
    'order': 'C',
    'file_offset': 0,
    'shape': (4813845, 385)}}}}

In [107]:
im_dict = rawio.signals_info_dict[(0,'imec0.ap')]

In [108]:
im_dict

{'fname': 'sglx_20250305-133614.imec0.ap',
 'meta': {'acqApLfSy': '384,0,1',
  'appVersion': '20230815',
  'fileCreateTime': '2025-03-05T13:36:14',
  'fileName': 'H:/data_neuropixel/20250305/20250305-133614/sglx_20250305-133614.imec0.ap.bin',
  'fileSHA1': '714C318109B5BCD7C52B19D2EEDDA3B3661D72D9',
  'fileSizeBytes': '3706660650',
  'fileTimeSecs': '160.4615',
  'firstSample': '404701412',
  'gateMode': 'Immediate',
  'imAiRangeMax': '0.62',
  'imAiRangeMin': '-0.62',
  'imAnyChanFullBand': 'true',
  'imCalibrated': 'true',
  'imChan0apGain': '100',
  'imDatApi': '3.62',
  'imDatBs_fw': '2.0.169',
  'imDatBsc_fw': '3.2.189',
  'imDatBsc_hw': '2.2',
  'imDatBsc_pn': 'NP2_QBSC_00',
  'imDatBsc_sn': '22491131',
  'imDatFx_hw': '0.1',
  'imDatFx_pn': 'NPM_FLEX_01',
  'imDatFx_sn': '23200177',
  'imDatHs_hw': '3.2',
  'imDatHs_pn': 'NPM_HS_31',
  'imDatHs_sn': '23200177',
  'imDatPrb_dock': '1',
  'imDatPrb_pn': 'NP2014',
  'imDatPrb_port': '3',
  'imDatPrb_slot': '5',
  'imDatPrb_sn': '22

In [98]:
#finding sync channel in nidq

for key in rawio.signals_info_dict[(0,'nidq')]['meta']:
    if 'sync' in key:
        print(key, rawio.signals_info_dict[(0,'nidq')]['meta'][key])

syncNiChan 2
syncNiChanType 0
syncNiThresh 1.1
syncSourceIdx 3
syncSourcePeriod 1


In [104]:
# Channel 0 in this data will have sync pulses at 1 Hz, let's confirm that
syncNiChan = int(rawio.signals_info_dict[(0,'nidq')]['meta']['syncNiChan'])
all_events = rawio.get_event_timestamps(0, 0, syncNiChan)
on_events = np.where(all_events[2] == f"XD{syncNiChan} ON")
on_ts = all_events[0][on_events]
on_ts_scaled = rawio.rescale_event_timestamp(on_ts)
on_diff = np.diff(on_ts_scaled)

In [106]:
on_diff

array([1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       1.00000841, 0.99998841, 1.00000841, 0.99998841, 1.00000841,
       0.99998841, 1.00000841, 1.00000841, 0.99998841, 1.00000841,
       0.99998841, 1.00000841, 0.99998841, 1.00000841, 1.00000841,
       0.99998841, 1.00000841, 0.99998841, 1.00000841, 0.99998

In [113]:
rawio.signals_info_dict[0, "imec0.ap"]['channel_names']

['AP0',
 'AP1',
 'AP2',
 'AP3',
 'AP4',
 'AP5',
 'AP6',
 'AP7',
 'AP8',
 'AP9',
 'AP10',
 'AP11',
 'AP12',
 'AP13',
 'AP14',
 'AP15',
 'AP16',
 'AP17',
 'AP18',
 'AP19',
 'AP20',
 'AP21',
 'AP22',
 'AP23',
 'AP24',
 'AP25',
 'AP26',
 'AP27',
 'AP28',
 'AP29',
 'AP30',
 'AP31',
 'AP32',
 'AP33',
 'AP34',
 'AP35',
 'AP36',
 'AP37',
 'AP38',
 'AP39',
 'AP40',
 'AP41',
 'AP42',
 'AP43',
 'AP44',
 'AP45',
 'AP46',
 'AP47',
 'AP48',
 'AP49',
 'AP50',
 'AP51',
 'AP52',
 'AP53',
 'AP54',
 'AP55',
 'AP56',
 'AP57',
 'AP58',
 'AP59',
 'AP60',
 'AP61',
 'AP62',
 'AP63',
 'AP64',
 'AP65',
 'AP66',
 'AP67',
 'AP68',
 'AP69',
 'AP70',
 'AP71',
 'AP72',
 'AP73',
 'AP74',
 'AP75',
 'AP76',
 'AP77',
 'AP78',
 'AP79',
 'AP80',
 'AP81',
 'AP82',
 'AP83',
 'AP84',
 'AP85',
 'AP86',
 'AP87',
 'AP88',
 'AP89',
 'AP90',
 'AP91',
 'AP92',
 'AP93',
 'AP94',
 'AP95',
 'AP96',
 'AP97',
 'AP98',
 'AP99',
 'AP100',
 'AP101',
 'AP102',
 'AP103',
 'AP104',
 'AP105',
 'AP106',
 'AP107',
 'AP108',
 'AP109',
 'AP110',


In [138]:
channel = 'SY0'
sync_data = rawio.get_analogsignal_chunk(channel_names = [channel], stream_index = stream_index)

In [139]:
# Convert the uint16 array to uint8
sync_data_uint8 = sync_data.view(np.uint8)

unpacked_sync_data = np.unpackbits(sync_data_uint8, axis=1)

In [140]:
this_stream = unpacked_sync_data[:,1]

In [141]:
timestamps, durations, labels = [], None, []

this_rising = np.where(np.diff(this_stream) == 1)[0] + 1
this_falling = (
    np.where(np.diff(this_stream) == 255)[0] + 1
)  # because the data is in unsigned 8 bit, -1 = 255!
if len(this_rising) > 0:
    timestamps.extend(this_rising)
    labels.extend([f"{channel} ON"] * len(this_rising))
if len(this_falling) > 0:
    timestamps.extend(this_falling)
    labels.extend([f"{channel} OFF"] * len(this_falling))
timestamps = np.asarray(timestamps)
if len(labels) == 0:
    labels = np.asarray(labels, dtype="U1")
else:
    labels = np.asarray(labels)

In [156]:
on_events = np.where(labels == f"{channel} ON")
on_ts = timestamps[on_events]
on_ts_scaled = on_ts / float(rawio.signals_info_dict[0, 'imec0.ap']['sampling_rate'])
on_diff = np.diff(on_ts_scaled)

In [158]:
on_diff

array([1.00003333, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.00003333, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.00003333, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.00003333, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.00003333, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.00003333, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

### DigitalChannelTest_g0

In [35]:
rawio_digital = SpikeGLXRawIO(testclass.get_local_path("spikeglx/DigitalChannelTest_g0"), load_sync_channel=True)
rawio_digital.parse_header()


In [36]:
for key in rawio_digital.signals_info_dict[(0,'nidq')]['meta']:
    if 'sync' in key:
        print(key, rawio_digital.signals_info_dict[(0,'nidq')]['meta'][key])

syncNiChan 0
syncNiChanType 0
syncNiThresh 1.1
syncSourceIdx 3
syncSourcePeriod 1


In [38]:
# Channel 0 in this data will have sync pulses at 1 Hz, let's confirm that
all_events = rawio_digital.get_event_timestamps(0, 0, 0)
on_events = np.where(all_events[2] == "XD0 ON")
on_ts = all_events[0][on_events]
on_ts_scaled = rawio_digital.rescale_event_timestamp(on_ts)
on_diff = np.diff(on_ts_scaled)

In [40]:
on_diff

array([1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 0.9996, 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
      